In [2]:
import pandas as pd
from datetime import datetime

rawpath = "../raw/"

In [4]:
df = pd.read_excel(f"{rawpath}/2022-ES-Tabelle.xlsx", names=['signatur','jahr','datum','medienart','sprache','land','koerperschaft_1','koerperschaft_2','person', 'titel','urheber_vorlage','ort','ort_idn','umfang','bemerkung'], dtype='string')
df = df.apply(lambda x: x.str.strip())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   signatur         82 non-null     string
 1   jahr             82 non-null     string
 2   datum            82 non-null     string
 3   medienart        82 non-null     string
 4   sprache          82 non-null     string
 5   land             82 non-null     string
 6   koerperschaft_1  82 non-null     string
 7   koerperschaft_2  10 non-null     string
 8   person           25 non-null     string
 9   titel            82 non-null     string
 10  urheber_vorlage  82 non-null     string
 11  ort              82 non-null     string
 12  ort_idn          82 non-null     string
 13  umfang           82 non-null     string
 14  bemerkung        21 non-null     string
dtypes: string(15)
memory usage: 9.7 KB


/home/andre/.cache/pypoetry/virtualenvs/base-_zdkNckr-py3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
def pica_schreiben(row):
    pica = f"""0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 {row.jahr}
1110 {row.datum}$4ezth
1130 !040445224!
1131 !040082407!
1132 a1-analog;a2-{row.medienart};f1-text;f2-blatt
1500 /1{row.sprache}
1700 /1{row.land}
3100 {row.koerperschaft_1}$BVerfasser$4aut
"""
    if pd.notna(row.koerperschaft_2):
        pica += f"3110 {row.koerperschaft_2}$BVerfasser$4aut\n"
    
    if pd.notna(row.person):
        pica += f"3010 {row.person}$BVerfasser$4aut\n"
    
    pica += f"""4000 {row.titel} / {row.urheber_vorlage}
4019 Geschäftsrundschreiben$Bobja
4046 {row.ort}
4060 {row.umfang}
4105 !1059146037!
4105 !1077661932!
"""
    if pd.notna(row.bemerkung):
        pica += f"4222 {row.bemerkung}\n"
    
    pica += f"""4700 |BSM|
5560 [RNA]Korrespondenz
5590 [Entstehungsort]
5590 {row.ort_idn}
5591 [Objektgattung]
5591 !042993512!
7001 {now.strftime('%d-%m-%y')} : x
4800 !009033645!
6710 !103243757X!
7100 {row.signatur} @ m
7109 !!DBSM/S!! ; {row.signatur}
\t\n"""
    return pica

In [4]:
now = datetime.now()
for index, row in df.sample(2).iterrows():
    print(pica_schreiben(row))

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1895
1110 27.08.1895$4ezth
1130 !040445224!
1131 !040082407!
1132 a1-analog;a2-druck;f1-text;f2-blatt
1500 /1ger
1700 /1XA-DXDE
3100 J. Veith$BVerfasser$4aut
3010 Wittwer, Konrad$BVerfasser$4aut
4000 Verkauf an Konrad Wittwer, der die Firma nach Stuttgart verlegt. / Konrad Wittwer
4019 Geschäftsrundschreiben$Bobja
4046 Stuttgart
4060 2 Exemplare
4105 !1059146037!
4105 !1077661932!
4700 |BSM|
5560 [RNA]Korrespondenz
5590 [Entstehungsort]
5590 !040582825!
5591 [Objektgattung]
5591 !042993512!
7001 08-09-22 : x
4800 !009033645!
6710 !103243757X!
7100 Bö-GR/V/76 @ m
7109 !!DBSM/S!! ; Bö-GR/V/76
	

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1875
1110 01.04.1875$4ezth
1130 !040445224!
1131 !040082407!
1132 a1-analog;a2-druck;f1-text;f2-blatt
1500 /1ger
1700 /1XA-DXDE
3100 C. Veith'sche Buchhandlung$BVerfasser$4aut
3010 Boller, Oscar$BVerfasser$4aut
4000 Verkauf

In [5]:
now = datetime.now()
with open(f"../dat/rundschreiben-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/rundschreiben-recent.dat", 'w') as f2:
        for index, row in df.iterrows():
            f.write(pica_schreiben(row))
            f2.write(pica_schreiben(row))

with open(f"../dat/rundschreiben-recent-sample.dat", 'w') as f:
    for index, row in df.sample(5).iterrows():
        f.write(pica_schreiben(row))